# Simple Geojson Sampler
Summary: Generates simple geojson into the `/data/generated`.

The content of those geojsons are mainly attribute data and timeseries data. 

First we prepare required imports for the whole scenario:

In [1]:
import sys
from pathlib import Path

# Add the parent directory to sys.path so 'core' can be imported
sys.path.append(str(Path.cwd().parent))


Now we need to add a map of required attributes. General attributes are just once per feature, timeseries per each year. 
Also there is need to set up timerange for timeseries per year.

In [2]:
from core.helpers_datetime import DatePeriod
from typing import Literal 

# Boundaries for the time series
timeseries_per_year_range = ("2000-01-01", "2025-01-01")

# Period step for the time series attribute records
timeseries_step = DatePeriod.MONTH

# Define the attributes for the GeoJSON features
# These attributes has no relation to the timeseries values
random_general_spatial_attributes: dict[str, Literal["string", "integer", "float"]] = {
    "general_average": "float",
    "general_median": "integer",
}

# These attributes are used to generate the timeseries data
# Each attribute will exist in versioon per year
random_timeseries_spatial_attributes: dict[str, Literal["string", "integer", "float"]] = {
    "population_density": "integer",
    "land_use": "string",
    "building_height": "float"
}


Now is the time to prepare a geojson. We use tool from this project:
- `geojson_features_generator` generates features with or without timeseries dimension.
- `geojson_feature_collection_body` wraps Geojson Features into the valid Geojson Feature Collection

Geometries will be random points generated inside the code

In [3]:
from core.geojson_generate import geojson_feature_collection_body, geojson_features_generator

# Create a simple GeoJSON FeatureCollection with one Point feature

# generator for GeoJSON features with timeseries data
timeseries_generator = geojson_features_generator(
    number_of_features=10, 
    attributes=random_timeseries_spatial_attributes,
    timeseries_attributes=random_timeseries_spatial_attributes,
    time_series_range=timeseries_per_year_range,
    timeseries_step=timeseries_step
)


# # iterate over the generators to create lists of features
# geojson_general_features: list = [feature for feature in general_generator]
geojson_timeseries_features: list = [feature for feature in timeseries_generator]

# Create the GeoJSON FeatureCollection body as final output
geojson_data = geojson_feature_collection_body(geojson_timeseries_features)


When we have Geojson data, we need to store output into the filesystem. This repository stores data into `/data/generated/` folder.

In [4]:
from core.io_store_data import write_geojson_to_generated_folder

# Store the generated GeoJSON data to the specified folder
write_geojson_to_generated_folder(
    geojson_data=geojson_data,
    folder_path=Path('../data/generated'),
    filename="timeseries_data.geojson"
)

And we are done. The geojson with timeseries should be present at `/data/generated/timeseries_data.geojson`